In [12]:
import cv2
import numpy as np
import serial
import time
import tkinter as tk
from PIL import Image, ImageTk
from collections import deque

esp = serial.Serial('COM5', 115200, timeout=1)
time.sleep(2)

color_ranges = {
    "dark_grey": (np.array([0, 0, 30]), np.array([180, 50, 120]))
  
}

cap = cv2.VideoCapture(1)
root = tk.Tk()
root.title("Color Tracker Using HSV")
root.geometry("700x500")

panel = tk.Label(root)
panel.pack()


buffer_size = 5
positions = deque(maxlen=buffer_size)

def update_frame():
    ret, frame = cap.read()
    if not ret:
        root.after(10, update_frame)
        return

    frame = cv2.flip(frame, 1)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    h, w, _ = frame.shape
    center_x = w // 2
    cv2.line(frame, (center_x, 0), (center_x, h), (0, 0, 255), 2)

    mask_total = np.zeros(hsv.shape[:2], dtype="uint8")

    for (lower, upper) in color_ranges.values():
        mask_total |= cv2.inRange(hsv, lower, upper)

    contours, _ = cv2.findContours(mask_total, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        c = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(c)

        if area > 800: 
            (x, y, cw, ch) = cv2.boundingRect(c)
            obj_x = x + cw // 2

            positions.append(obj_x)
            smooth_x = int(np.mean(positions))

            cv2.rectangle(frame, (x, y), (x+cw, y+ch), (0, 255, 0), 2)
            cv2.circle(frame, (smooth_x, y + ch // 2), 5, (255, 0, 0), -1)

            deadzone = 40
            if smooth_x < center_x - deadzone:
                esp.write(b"L")   
            elif smooth_x > center_x + deadzone:
                esp.write(b"R") 
            else:
                esp.write(b"C") 

    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img)
    imgtk = ImageTk.PhotoImage(image=img)
    panel.imgtk = imgtk
    panel.config(image=imgtk)

    root.after(30, update_frame)  

def on_closing():
    cap.release()
    esp.close()
    root.destroy()

root.protocol("WM_DELETE_WINDOW", on_closing)
update_frame()
root.mainloop()

In [1]:
xn = list(map(int, input("Enter sequence xn (comma-separated): ").split(",")))
yn = list(map(int, input("Enter sequence yn (comma-separated): ").split(",")))

origin_xn = int(input("Enter origin of xn: "))
origin_yn = int(input("Enter origin of yn: "))

if origin_xn > origin_yn:
    diff = origin_xn - origin_yn
    yn = [0]*diff + yn
    origin_yn += diff
elif origin_yn > origin_xn:
    diff = origin_yn - origin_xn
    xn = [0]*diff + xn
    origin_xn += diff

last_xn = len(xn) - origin_xn
last_yn = len(yn) - origin_yn
diff_last = abs(last_xn - last_yn)

if last_xn > last_yn:
    yn = yn + [0] * diff_last
else:
    xn = xn + [0] * diff_last

print("Final aligned sequences:")
print("origin of xn:", origin_xn)
print("origin of yn:", origin_yn)
print("xn:", xn)
print("yn:", yn)

Final aligned sequences:
origin of xn: 554
origin of yn: 554
xn: [41, 0, 0, 0, 0, 0, 0, 0, 0, 0]
yn: [0, 0, 0, 0, 0, 0, 0, 0, 0, 65]
